In [79]:
from brickschema import Graph

Load the building model:

In [80]:
g = Graph(load_brick=True)
g.load_file("../../datasets/bts_site_b_train/Site_B_cleaned.ttl")
print(f"Building model has {len(g)} triples")

Building model has 52457 triples


Expand the building model through inference:

In [81]:
g.expand(profile="rdfs") # inference using RDFS reasoning
print(f"Expanded building model has {len(g)} triples")

Expanded building model has 89052 triples


In [82]:
g.expand(profile="shacl") # infererence using Brick classes from Brick tags
print(f"Expanded graph has {len(g)} triples")

2024-10-07:20:43:17,629 WARNING [graph.py:293] Validation Report
Conforms: False
Results (24):
Constraint Violation in OrConstraintComponent (http://www.w3.org/ns/shacl#OrConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ rdf:type rdfs:Resource ; sh:or ( [ rdf:type rdfs:Resource ; sh:class brick:Room ] [ rdf:type rdfs:Resource ; sh:class brick:Space ] [ rdf:type rdfs:Resource ; sh:class brick:Zone ] ) ; sh:path brick:hasPart ]
	Focus Node: p33f3e0c2_f2cd_471c_b5a0_4655c2bd4623:e8e7705d_b87a_4bf3_b454_202ca27ea7ad
	Value Node: p33f3e0c2_f2cd_471c_b5a0_4655c2bd4623:d02c6fdd_ce6b_44a1_b193_1ede85e7d4c8.6d18ee1e_22f5_495f_89e6_cd613015478e
	Result Path: brick:hasPart
	Message: Node p33f3e0c2_f2cd_471c_b5a0_4655c2bd4623:d02c6fdd_ce6b_44a1_b193_1ede85e7d4c8.6d18ee1e_22f5_495f_89e6_cd613015478e does not conform to one or more shapes in [ rdf:type rdfs:Resource ; sh:class brick:Room ] , [ rdf:type rdfs:Resource ; sh:class brick:Space ] , [ rdf:type rdfs:Resource ; sh:class brick:Zo

Expanded graph has 94491 triples


Play around with some queries:

In [90]:
# Get all tags for the building
query = """
    SELECT ?tag WHERE {
        p33f3e0c2_f2cd_471c_b5a0_4655c2bd4623:13211186_beb4_4227_bd2d_0644e860886e brick:hasTag ?tag .
    }
"""

# Get all entities that have the tags Air and Temperature
# query = """
#     SELECT ?entity WHERE {
#         ?entity  brick:hasTag tag:Air, tag:Temperature .
#     }
# """

# Get all classes that are associated with the tags Air and Temperature
# query = """
# SELECT ?class WHERE {
#     ?class  brick:hasAssociatedTag tag:Air, tag:Temperature .
# }
# """

# Get all classes that are associated with the tag Room
# query = """
# SELECT ?class WHERE {
#     ?class  brick:hasAssociatedTag tag:Room .
# }
# """

res = g.query(query)

for row in res:
    print(row)

(rdflib.term.URIRef('https://brickschema.org/schema/BrickTag#Building'),)
(rdflib.term.URIRef('https://brickschema.org/schema/BrickTag#Location'),)


In [89]:
# Get all classes that are associated with the tag Location
query = """
SELECT ?class WHERE {
    ?class  brick:hasAssociatedTag tag:Location .
}
"""

res = g.query(query)

for row in res:
    print(row)

(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Ablutions_Room'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Atrium'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Auditorium'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Basement'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Battery_Room'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Bench_Space'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Broadcast_Room'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Cafeteria'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Cold_Box'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Concession'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Conference_Room'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Control_Room'),)
(rdflib.term.URIRef('https://brickschema.org/schema/Brick#Copy_Room'),)
(rdflib.term.URIRef('https://brickschema.or

In [91]:
# Get the source and stream of all data streams
res = g.query("""
    SELECT ?src ?sid WHERE  {
        ?src senaps:stream_id ?sid .
    }
""")
for row in res:
    print(row)

(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#da5873de_e112_4f9b_9e0f_149bd1221894.8084c259_152b_4607_808b_0fd0a92973e2.d851e5bd_1766_4aae_a571_d5070bcc3493'), rdflib.term.Literal('3dfa2bab_f8f2_485b_80e9_5afdeeac9dc4'))
(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#da5873de_e112_4f9b_9e0f_149bd1221894.8084c259_152b_4607_808b_0fd0a92973e2.04fa8942_55b0_41fb_9cfc_66d087d70245'), rdflib.term.Literal('77ed01a5_11c3_4607_9bf8_342b947c88de'))
(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#da5873de_e112_4f9b_9e0f_149bd1221894.8084c259_152b_4607_808b_0fd0a92973e2.df34251f_78ab_4f7b_8e8b_75ed453cbe89'), rdflib.term.Literal('16b768f0_5fb1_4db3_b506_d811b7c67738'))
(rdflib.term.

Get all the IDs of all conference rooms and their related air temperature sensors and setpoints:

In [92]:
conference_rooms = g.query("""SELECT ?loc ?ats ?atsp WHERE  {
    ?ats    a       brick:Air_Temperature_Sensor .
    ?atsp   a       brick:Room_Air_Temperature_Setpoint .
    ?ats    brick:isPointOf ?loc .
    ?atsp   brick:isPointOf ?loc .
    ?loc    a   brick:Conference_Room
}""")
for row in conference_rooms:
    print(row)

(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#d02c6fdd_ce6b_44a1_b193_1ede85e7d4c8.89e05278_e33b_4bac_b51b_1204446d5017'), rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#8dbdff35_af3a_4c69_8963_53bdcb33c775.c759ad9f_9d6d_45c1_ad1c_377840198f64'), rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#8dbdff35_af3a_4c69_8963_53bdcb33c775.418326e3_98a5_4cb5_8e24_160c3d33b813'))
(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#d02c6fdd_ce6b_44a1_b193_1ede85e7d4c8.0e964c7f_9275_47df_97c5_ede973dc0767'), rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8

Get the ID(s) of any outside air temperature sensor(s) associated with a weather station:

In [94]:
weather_station = g.query("""SELECT ?ats WHERE  {
    ?ats    a       brick:Outside_Air_Temperature_Sensor .
    ?ats    brick:isPointOf ?loc .
    ?loc    a   brick:Weather_Station
}""")
for row in weather_station:
    print(row)

(rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#1ca047e2_b601_4ba7_a579_abcfa7a45ea7.a48e2b92_9b73_4fbf_99d7_3fa6824903ff'),)


In [95]:
print(type(weather_station))
e = list(weather_station)[0]
print(type(e))
print(e[0])

<class 'rdflib.plugins.sparql.processor.SPARQLResult'>
<class 'rdflib.query.ResultRow'>
dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#1ca047e2_b601_4ba7_a579_abcfa7a45ea7.a48e2b92_9b73_4fbf_99d7_3fa6824903ff


Get the stream ID of the outside air temperature sensor:

In [ ]:
# outside_temperature_stream_query = f"""
#     SELECT ?sid WHERE  {
#         dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#1ca047e2_b601_4ba7_a579_abcfa7a45ea7.a48e2b92_9b73_4fbf_99d7_3fa6824903ff senaps:stream_id ?sid
#     }
# """
# outside_temperature_stream_query = f"""
# SELECT ?sid WHERE  {{
#     <{weather_station}> senaps:stream_id ?sid
# }}
# """
outside_temperature_stream_query = f"""
SELECT ?sid WHERE  {{
    VALUES ?ats {{ {weather_station[0]} }}
    ?ats senaps:stream_id ?sid
}}
"""
print(repr(outside_temperature_stream_query))
import rdflib
outside_temperature_stream = g.query(outside_temperature_stream_query, initBindings={'weather_station': rdflib.URIRef(weather_station[0])})
for row in outside_temperature_stream:
    print(row)

In [ ]:
outside_temperature_stream = g.query("""
    SELECT ?ats ?sid WHERE  {                      
        SELECT ?ats WHERE  {
            ?ats    a                  brick:Outside_Air_Temperature_Sensor .
            ?ats    brick:isPointOf    ?loc .
            ?loc    a                  brick:Weather_Station .
        }
        ?ats senaps:stream_id ?sid .
    }
""")
for row in weather_station:
    print(row)

In [96]:
for row in weather_station:
    # print(row)
    outside_temperature_stream_query = """
        SELECT ?ats ?sid WHERE  {
            ?ats senaps:stream_id ?sid
        }
    """
    outside_temperature_stream = g.query(outside_temperature_stream_query, initBindings={'ats': row[0]})

    # for row in outside_temperature_stream:
    #     print(row)
    print(outside_temperature_stream.bindings)
    binding = outside_temperature_stream.bindings[0]
    print(binding)
    print(binding['sid'])
    

[{rdflib.term.Variable('ats'): rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#1ca047e2_b601_4ba7_a579_abcfa7a45ea7.a48e2b92_9b73_4fbf_99d7_3fa6824903ff'), rdflib.term.Variable('sid'): rdflib.term.Literal('664ddc27_ccc5_43a1_97b7_3000c0e948db')}]
{rdflib.term.Variable('ats'): rdflib.term.URIRef('dch:org/df125f8d_93e4_4a4f_bc87_1c4e332f32b8/site/c495fbd8_7a71_44b4_ad2a_014ac2df7ce9/building/95d1f7f3_33c6_4bd8_a82c_99c550d83bb8#1ca047e2_b601_4ba7_a579_abcfa7a45ea7.a48e2b92_9b73_4fbf_99d7_3fa6824903ff'), rdflib.term.Variable('sid'): rdflib.term.Literal('664ddc27_ccc5_43a1_97b7_3000c0e948db')}
664ddc27_ccc5_43a1_97b7_3000c0e948db
